## 模型的保存与恢复

模型的处理过程可以保存下来。

当发布模型时，我们需要共享的是：

- 创建模型的代码 code
- 模型训练的参数

**注意**, 因为保存下来的是模型代码，所以代码要注意安全问题。

### 数据集准备

本例中，我们使用 Keras 来对 mnist 数据集进行训练。


In [9]:
import os
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)


2.0.0-alpha0


In [10]:
mnist = keras.datasets.mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0



In [24]:
len(test_labels)
set(train_labels)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

### 模型定义

In [26]:
# Returns a short sequential model
def create_model():
    model = keras.Sequential([
        keras.layers.Dense(
            512,
            activation = tf.nn.relu,
            input_shape = (784,)
        ),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation = tf.nn.softmax)
    ])
    
    model.compile(
        optimizer = keras.optimizers.Adam(),
        loss = keras.losses.sparse_categorical_crossentropy,
        metrics = ['accuracy']
    )
    
    return model
    

In [27]:
model = create_model()
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### 保存检查点, checkpoints 

最常见的情况就是在训练过程中和训练结束后保存相应的检查点，checkpoints。这样在使用模型时就不需要重新训练了，而且训练中断还可以从检查点中恢复。

#### 检查点回调的使用

在训练的过程中，传递一个 `ModelCheckpoint` 回调即可。

In [29]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    save_weights_only = True,
    verbose = 1
)

model = create_model()
model.fit(
    train_images,
    train_labels,
    batch_size = 128,
    epochs = 10,
    validation_data = (test_images, test_labels),
    callbacks = [cp_callback]
)

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 128/1000 [==>...........................] - ETA: 0s - loss: 2.3501 - accuracy: 0.1250
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 324us/sample - loss: 1.7311 - accuracy: 0.5070 - val_loss: 1.2325 - val_accuracy: 0.7330
Epoch 2/10
 896/1000 [=========================>....] - ETA: 0s - loss: 0.8324 - accuracy: 0.8192
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 125us/sample - loss: 0.8069 - accuracy: 0.8230 - val_loss: 0.7769 - val_accuracy: 0.7840
Epoch 3/10
 640/1000 [==================>...........] - ETA: 0s - loss: 0.5237 - accuracy: 0.8641
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 161us/sample - loss: 0.4968 - accuracy: 0.8720 - val_loss: 0.6021 - val_accuracy: 0.8210
Epoch 4/10
 640/1000 [==================>...........] - ETA: 0s - loss: 0.3529 - accuracy: 0.9094
Epoch

In [32]:
!ls -lh {checkpoint_dir}

total 9568
-rw-r--r--  1 eason  staff    71B Jun 26 17:11 checkpoint
-rw-r--r--  1 eason  staff   4.7M Jun 26 17:11 cp.ckpt.data-00000-of-00001
-rw-r--r--  1 eason  staff   1.5K Jun 26 17:11 cp.ckpt.index


现在，我们就可以再创建一个新的模型通过加载保存的参数即可恢复。

In [36]:
model = create_model()

# 一个未经训练后的模型
loss, acc = model.evaluate(test_images, test_labels)
print('Untrained model, accuracy: {:5.2f}%'.format(100 * acc))

# 通过加载保存参数后的模型
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

1000/1000 [==============================] - 0s 105us/sample - loss: 2.3328 - accuracy: 0.0870
Untrained model, accuracy:  8.70%
1000/1000 [==============================] - 0s 58us/sample - loss: 0.4097 - accuracy: 0.8600
Restored model, accuracy: 86.00%


### 检查点的选项

检查点还可配置相应的名称和检查点的名称。下面我们就训练一个新模型，并且每经过 5 个 epochs 保存一个新的检查点文件。


In [45]:
# include the epoch in the file name. (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path,
    verbose = 1,
    save_weights_only = True,
    period = 5 # Every 5-epochs we save a new file.
)

model = create_model()
model.save_weights(checkpoint_path.format(epoch = 0))

model.fit(
    train_images,
    train_labels,
    epochs = 50,
    callbacks = [cp_callback],
    validation_data = (test_images, test_labels),
    verbose = 0
)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [46]:
!ls {checkpoint_dir}

checkpoint                       cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001 cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index               cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001 cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index               cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001 cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index               cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001 cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index               cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001 cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index               cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


测试一下我们的新的保存点。默认情况下 tensorflow 只保存最近的5个检查点。

In [48]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

model = create_model()
model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels)
print('Restore model, accuracy: {:5.2f}%'.format(100 * acc))

training_2/cp-0050.ckpt
1000/1000 [==============================] - 0s 123us/sample - loss: 0.4856 - accuracy: 0.8780
Restore model, accuracy: 87.80%


### 这些文件是什么？

上面的文件保存的训练权重的二进制形式。 data 文件保存的是数据，而 index 文件的是数据保存的索引文件。

### 手动保存 weights 

调用 `save_weights()` 方法也可以自己手动保存检查点。

In [49]:
# save the weights
model.save_weights('./checkpoints/my_checkpoint')

# restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels)
print('Restore model, accuracy: {:5.2f}%'.format(100 * acc))


1000/1000 [==============================] - 0s 120us/sample - loss: 0.4856 - accuracy: 0.8780
Restore model, accuracy: 87.80%


### 保存整个模型

上面我们只是保存了训练后的参数，我们在使用的时候还需要调用 `create_model()` 即原来的模型代码。如果我们将整个模型存储下来，那么我们就不保存代码了。我们需要保存权重参数，模型配置，甚至是 optimizer 的配置。

将整个模型保存下来会非常有用，这样其它语文的 API 也可以使用它。如 TensorFlow.js。

#### 将模型保存为 HDF5 文件

Keras 提供将模型保存为 HDF5 标准的文件。保存后的模型文件可以看成仅仅是一个二进制文件。

In [50]:
model = create_model()

model.fit(
    train_images, 
    train_labels,
    epochs = 10
)

# Save entire model to a HDF5 file.
model.save('my_model.h5')

Epoch 1/10
1000/1000 [==============================] - 0s 222us/sample - loss: 1.1920 - accuracy: 0.6550
Epoch 2/10
1000/1000 [==============================] - 0s 125us/sample - loss: 0.4161 - accuracy: 0.8760
Epoch 3/10
1000/1000 [==============================] - 0s 150us/sample - loss: 0.2946 - accuracy: 0.9150
Epoch 4/10
1000/1000 [==============================] - 0s 132us/sample - loss: 0.2169 - accuracy: 0.9480
Epoch 5/10
1000/1000 [==============================] - 0s 140us/sample - loss: 0.1520 - accuracy: 0.9700
Epoch 6/10
1000/1000 [==============================] - 0s 136us/sample - loss: 0.1205 - accuracy: 0.9830
Epoch 7/10
1000/1000 [==============================] - 0s 129us/sample - loss: 0.0899 - accuracy: 0.9880
Epoch 8/10
1000/1000 [==============================] - 0s 130us/sample - loss: 0.0703 - accuracy: 0.9920
Epoch 9/10
1000/1000 [==============================] - 0s 132us/sample - loss: 0.0505 - accuracy: 0.9970
Epoch 10/10
1000/1000 [=======================

In [53]:
!ls -lh my_model.h5 

-rw-r--r--  1 eason  staff   4.7M Jun 26 17:56 my_model.h5


#### 加载模型

In [55]:
# Recreate the exact same model, including weights and optimizer

new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_36 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_18 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [56]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 107us/sample - loss: 0.4182 - accuracy: 0.8650
Restored model, accuracy: 86.50%


#### 作为 tf.keras 的模型保存, As a saved_model

**注意**, tf.keras model 是实验性的，后面的版本可能会变化。 从下面的信息就可以出 API 又换了。

In [57]:
model = create_model()
model.fit(train_images, train_labels, epochs = 5)

saved_model_path = tf.contrib.saved_model.save_keras_model(
    model, 
    './saved_models'
)

Epoch 1/5
1000/1000 [==============================] - 0s 293us/sample - loss: 1.1778 - accuracy: 0.6720
Epoch 2/5
1000/1000 [==============================] - 0s 168us/sample - loss: 0.4316 - accuracy: 0.8700
Epoch 3/5
1000/1000 [==============================] - 0s 155us/sample - loss: 0.2821 - accuracy: 0.9260
Epoch 4/5
1000/1000 [==============================] - 0s 141us/sample - loss: 0.2009 - accuracy: 0.9550
Epoch 5/5
1000/1000 [==============================] - 0s 226us/sample - loss: 0.1617 - accuracy: 0.9680


AttributeError: module 'tensorflow' has no attribute 'contrib'